## Warning!

This example notebook is not up-to-date with the latest version of Plotly's Python API (version 1.0.\*). 

Refer to Plotly's Python [User Guide](http://nbviewer.ipython.org/github/plotly/python-user-guide/blob/master/s00_homepage/s00_homepage.ipynb) 
and more specifically [section 7.3](http://nbviewer.ipython.org/github/plotly/python-user-guide/blob/master/s7_streaming/s7_streaming_p3-bubbles.ipynb) for an updated version of this notebook.

<hr>

### Recreating Hans Rosling's Famous Animated Bubble Charts <br>  with Plotly Real-time Streaming Graphs

Learn more about real-time streaming graphs with plotly here:

[https://github.com/plotly/streaming-demos](https://github.com/plotly/streaming-demos)

In [14]:
import chart_studio.plotly as plotly
import chart_studio
import requests
import time
import json
import math
import pprint

In [8]:
# Fill in the config.json file in this directory with your plotly username, 
# plotly API key, and your generated plotly streaming tokens
# Sign up to plotly here: https://plot.ly/ssu
# View your API key and streaming tokens here: https://plot.ly/settings

username = 'Parrotlife'
api_key = 'HVithFEqzxhtolNyW0jU'
stream_token = '49wpntpsv8'

In [17]:
# Initialize your plotly object
chart_studio.tools.set_credentials_file(username='Parrotlife', api_key='HVithFEqzxhtolNyW0jU')


In [18]:
# Embed the stream token in one of the traces of a plotly-data object - one token per trace
axes = {
    'showgrid': False,
    'showline': False,
    'ticks': '',
    'zeroline': False,
    'titlefont': {'size': 16, 'color': 'rgb(67,67,67)'},
    'tickfont': {'size': 13, 'color': 'rgb(67,67,67)'}
}
xaxis = dict({'range': [0, 5e4], 'type': 'log', 'title': 'GDP per Capita'}.items() + axes.items())
yaxis = dict({'range': [30, 83], 'title': 'Life Expectancy', 'exponentformat': 'power'}.items() + axes.items())

r = p.plot([{
            'x':[],
            'y':[],
            'text':[],
            'mode': 'markers',
            'marker': {
                'sizemode': 'area',
                'sizeref': 200000,#0.05,
                'opacity': 0.55,
                'line': {
                    'width': 1,
                    'color': 'white'
                }
            },
            'showlegend': False,
            'stream': {
                'token': stream_token, 'maxpoints': 29
            }},
        ],
    layout={
        'xaxis': xaxis,
        'yaxis': yaxis,
        'titlefont':{'size': 32, 'color': 'rgb(67,67,67)'},
        'legend': {
            'bgcolor': 'rgba(255, 255, 255, 0)',
            'bordercolor': 'rgba(255, 255, 255, 0)',
            'fontcolor': 'rgb(67, 67, 67)',
            'size': 13,
            'x': 100,
            'y': 0.95
        }  
    },
    filename='Hans Rosling Streaming', fileopt='overwrite', width=1000, height=700)

pprint.pprint(r)

TypeError: unsupported operand type(s) for +: 'dict_items' and 'dict_items'

In [5]:
# Let's load up the Hans Rosling Gapminder Data
# It's saved as a JSON file in a gist, you can check out the file here: https://gist.github.com/chriddyp/9167305
def readjson(url):
    import urllib2, StringIO, json
    response = urllib2.urlopen(url).read()
    data = json.loads(response)
    return data

data = readjson("https://gist.githubusercontent.com/chriddyp/9167305/raw/db58a28b2144121cbf694ab1b8f73b142b96a464/gapminder.json")

In [6]:
# Let's just take a quick look at the JSON file
# The keys are country names:
print data.keys()[0:10]

[u'Canada', u'Brazil', u'Sao Tome and Principe', u'Guernsey', u'Turkmenistan', u'Saint Helena', u'Serbia and Montenegro', u'Ireland', u'Lithuania', u'Cambodia']


In [7]:
# And for each country, we have historical data about the GDP, Life Expectancy and Population. 
# Also, the continent/region that the country belongs to
print data['Canada'].keys()

[u'gdp', u'lifeExp', u'continent', u'pop', u'years']


In [8]:
# The data is just stored as arrays, here are some values:
print data['Canada']['gdp'][0:10]

[10581.265520182, 10932.4667838283, 11367.1611248765, 11586.6145455354, 11173.2594978109, 11901.5094889284, 12555.5494657206, 12489.9500550468, 12384.4101756439, 12590.8003302521]


In [9]:
# interpolate the raw data - insert 3 values between every year
import numpy as np
years = np.linspace(1950, 2007, (2007-1950)*1+1)
for country in data:
    if len(data[country]['years'])>0:
        x = np.linspace(data[country]['years'][0], 
                        data[country]['years'][-1], 
                        (data[country]['years'][-1] - data[country]['years'][0])*12+1)
        data[country]['gdp'] = np.interp(x, data[country]['years'], data[country]['gdp'])
        data[country]['lifeExp'] = np.interp(x, data[country]['years'], data[country]['lifeExp'])
        data[country]['pop'] = np.interp(x, data[country]['years'], data[country]['pop'])    
        data[country]['years'] = x        

In [10]:
# To stream data, initialize a plotly streaming object with a streaming token and write to it
# In this case, we'll yield an object in the form:
# {'x': [...], 'y': [...], 'text': [...], 'marker': {'size': [...], 'color': [...]}}
# Where the 'x' data is each country's GDP
#           'y' data is each country's Life Expectancy
#           'text' data is each country's name
#           'size' data is a scaled version of the country's population
#           'color' data is color coded to the continents
# Each data object will correspond to a specific year, and we'll "yield" a new object each year
# This data object will stream to to our plotly graph, updating it in real-time!

s = plotly.stream(stream_token)

yearCnt = 0
continent_colors = {'Asia': '#1b9e77', 
    'Middle East, North Africa, and Greater Arabia': '#d95f02', 
    'Europe': '#7570b3', 
    'North America': '#e7298a',
    'Central America and the Caribbean': '#66a61e',
    'South America': '#e6ab02',
    'Sub-Saharan Africa': '#a6761d',
    'Oceania': '#666666'}
while True:
    year = years[yearCnt%len(years)]
    graph = {   'x': [],
                'y': [],
                'text': [],
                'marker': {
                    'size': [],
                    'color': []
                }
            }

    for country in data:
        # find the index of the year
        i = np.where(data[country]['years']==year)[0]
        if len(i)>0:
            graph['x'].extend(data[country]['gdp'][i])
            graph['y'].extend(data[country]['lifeExp'][i])
            graph['marker']['size'].extend(data[country]['pop'][i])
            graph['text'].append(country)
            graph['marker']['color'].append(continent_colors[data[country]['continent']])

    if year%1 == 0:
        graph['layout'] = {'title': str(int(year))}

    s.write(graph)
    time.sleep(0.75)
    yearCnt+=1

In [11]:
# We can view the graph in the web browser, or embedded as an IFrame inside the notebook
plotly.display(r['url'])

### Give it a try yourself!

##### More Streaming IPython Notebooks:

[http://nbviewer.ipython.org/github/plotly/Streaming-Demos/tree/master/IPython%20examples/](http://nbviewer.ipython.org/github/plotly/Streaming-Demos/tree/master/IPython%20examples/)

##### More about Plotly Streaming 

[https://github.com/plotly/Streaming-Demos](https://github.com/plotly/Streaming-Demos)

##### Get in touch!

[http://twitter.com/plotlygraphs](@plotlygraphs)

[https://facebook.com/plotly](https://facebook.com/plotly)

<ben@plot.ly>

In [4]:
# CSS styling within IPython notebook
from IPython.core.display import HTML
import urllib2
def css_styling():
    url = 'https://raw.githubusercontent.com/plotly/python-user-guide/master/custom.css'
    styles = urllib2.urlopen(url).read()
    return HTML(styles)

css_styling()